In [1]:
# imported dependencies
import pennylane as qml
from itertools import product
import numpy as np
import random
qml.version()

'0.28.0'

In [2]:
dev = qml.device("default.mixed", wires=5,shots=1)

In [27]:
# n = 5 (Physical Qubits)
# k = 3 (Logical Qubits)

def generate_density_matrix():

    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([c1, 0, 0, 0, 0, 0, 0, c2])
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

In [28]:
@qml.qnode(dev)
def syndrome_measurement(density_matrix, estimation_wires,target_wires):
    
    # should be a density matrix
    qml.QubitDensityMatrix(density_matrix, wires=target_wires)
    
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    
    # Introduce Bit Error
    qml.BitFlip(0.75,target_wires[0])

    #Detects if bit 0 or bit 1 flipped
    g1_kron = qml.PauliZ(target_wires[0]) @ qml.PauliZ(target_wires[1]) @ qml.Identity(wires=target_wires[2])

    # Detect if bit 1 or bit 2 flipped
    g2_kron = qml.Identity(wires=target_wires[0]) @ qml.PauliZ(target_wires[1]) @ qml.PauliZ(target_wires[2])
    
    g1_kron = g1_kron.matrix()
    g2_kron = g2_kron.matrix()
    
    # Apply generators on ancilla bits
    qml.ControlledQubitUnitary(g1_kron, control_wires=estimation_wires[0], wires=target_wires)
    qml.ControlledQubitUnitary(g2_kron, control_wires=estimation_wires[1], wires=target_wires)

    return qml.density_matrix(target_wires)

In [35]:
estimation_wires = [0, 1]
target_wires = [2,3,4]
rho_id = generate_density_matrix()

print(qml.draw(syndrome_measurement)(rho_id,estimation_wires,target_wires))

0: ──H─────────────────────────────────────╭●────────────┤       
1: ──H─────────────────────────────────────│──────╭●─────┤       
2: ─╭QubitDensityMatrix(M0)──BitFlip(0.75)─├U(M1)─├U(M2)─┤ ╭State
3: ─├QubitDensityMatrix(M0)────────────────├U(M1)─├U(M2)─┤ ├State
4: ─╰QubitDensityMatrix(M0)────────────────╰U(M1)─╰U(M2)─┤ ╰State


In [36]:

def density_matrix_to_state(rho):
    # Compute the eigenvectors and eigenvalues of the density matrix
    eigvals, eigvecs = np.linalg.eig(rho)
    # Take the square root of the eigenvalues
    sqrt_eigvals = np.sqrt(eigvals)
    # Multiply each eigenvector by the square root of the corresponding eigenvalue
    state_vectors = [np.multiply(sqrt_eigvals[i], eigvecs[:, i]) for i in range(len(eigvals))]
    # Add up all the resulting vectors to get the state vector
    state_vector = sum(state_vectors)
    # Normalize the state vector
    state_vector /= np.linalg.norm(state_vector)
    return state_vector

rho = syndrome_measurement(rho_id, estimation_wires,target_wires)
st = density_matrix_to_state(rho)
print("st")
print(st)

st
[0.35355339+0.j 0.        +0.j 0.        +0.j 0.61237244+0.j
 0.61237244+0.j 0.        +0.j 0.        +0.j 0.35355339+0.j]


In [37]:
#Detects if bit 0 or bit 1 flipped
g1_kron = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)
g1_kron = g1_kron.matrix()

# Detect if bit 1 or bit 2 flipped
g2_kron = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
g2_kron = g2_kron.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()
p1 = 0.5 * (I + g1_kron)
p2 = 0.5 * (I + g2_kron)

P = p1 @ p2
num = P @ rho @ P

# Expectation value of P 
denom = np.trace(rho @ P)

rho_det = num/denom

print('rho_id')
print(rho_id)

rho_id
[[0.5 0.  0.  0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.  0.  0.  0.5]]


In [38]:
print("rho (with noise)")
print(rho)

rho (with noise)
[[0.125+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.125+0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.375+0.j 0.375+0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.375+0.j 0.375+0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.   +0.j]
 [0.125+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
  0.125+0.j]]


In [39]:
print("rho_det")
print(Pdet)


rho_det
[[0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j]]


In [34]:
print(np.allclose(rho_id,rho_det))

True
